## Análisis SST Región Niño 3.4

**PROYECTO:** SISTEMA PARA EL SEGUIMIENTO DE ECOSISTEMAS VENEZOLANOS \
**AUTOR:** Javier Martinez

In [1]:
from MONGO import CONEXION
from datetime import datetime
import pandas as pd

In [2]:
import locale

# Cambiando sistema horario
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

'es_ES.UTF-8'

# Creando Coenxión con Mongo DB

In [3]:
# Creando la conexión con MongoDB
db = CONEXION.conexion()
db.list_collection_names()

['polygons', 'meteorological', 'SSTNino34']

# Descargando la Información SST NIÑO3.4

In [4]:
# Fecha actual
time = datetime.today().toordinal()

# Realizando consulta
sst_data = db.SSTNino34.find({"time":{"$lte":time}})

# Generando pandas dataframe
data_pandas = pd.DataFrame([file for file in sst_data])
data_pandas['periodo'] = data_pandas.time.apply(lambda x: datetime.fromordinal(x))
data_pandas['mes_year'] =  data_pandas['periodo'].dt.strftime('%B-%Y')
data_pandas.index = pd.to_datetime(data_pandas.periodo)
data_pandas.head()

,_id,year,month,nino34_mean,climatologica,anomalias,mes,time,time_actualizacion,periodo,mes_year
periodo,,,,,,,,,,,
2022-06-01,62e6d1d870bebdf1983a27a9,2022,6,26.96,27.73,-0.77,junio,738307,738367,2022-06-01,junio-2022
2022-05-01,62e6d1d870bebdf1983a27a8,2022,5,26.82,27.94,-1.12,mayo,738276,738367,2022-05-01,mayo-2022
2022-04-01,62e6d1d870bebdf1983a27a7,2022,4,26.71,27.83,-1.12,abril,738246,738367,2022-04-01,abril-2022
2022-03-01,62e6d1d870bebdf1983a27a6,2022,3,26.32,27.29,-0.98,marzo,738215,738367,2022-03-01,marzo-2022
2022-02-01,62e6d1d870bebdf1983a27a5,2022,2,25.87,26.76,-0.90,febrero,738187,738367,2022-02-01,febrero-2022


In [5]:
# Registros
print(data_pandas.shape)

(870, 11)


In [6]:
# Data disponible
print(data_pandas.index.min())
print(data_pandas.index.max())

1950-01-01 00:00:00
2022-06-01 00:00:00


### Estudio de la serie temporal

In [18]:
# Serie de tiempo
def temperatura_nino34_plot(data_model=data_pandas.dropna().copy()):
    
    import plotly.graph_objects as go
    from plotly.graph_objects import Layout

    fig = go.Figure(layout=Layout(plot_bgcolor='rgba(0,0,0,0)'))
    fig.add_trace(go.Scatter(x=data_model.index, y=data_model['nino34_mean'],
                             mode='lines',name='NIÑO 3.4',
                             line=dict(color='#C10101', width=2),
                             xperiodalignment="middle"
                            ))

    fig.update_xaxes(tickformat="%Y/%m",
                     showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                     ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10
                    )
    
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                     ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)

    fig.update_traces(textfont_size=18)
    fig.update_layout(title='Temperatura promedio en la región NIÑO 3.4',
                      xaxis_title='Mes',
                      yaxis_title='Temperatura (°C)',
                      legend_title_text='Serie',
                      legend_title = dict( font = dict(size = 30)),
                      legend=dict(y=0.5,
                                  traceorder='reversed',
                                  font_size=11),

                       uniformtext_minsize=8,
                       uniformtext_mode='hide',
                       height=800,
                       width=1500,
                       font = dict(size = 22),
                       xaxis_range=['1949-01-01','2023-04-01']
                     )

    return fig
    
fig_sstt_nino34 = temperatura_nino34_plot()
fig_sstt_nino34.write_image('./figure/nino34/sstt_nino34.png')

fig_sstt_nino34.show()

In [19]:
# Descomposicion de la serie de tiempo
def seasonal_decompose_plot(data=data_pandas.dropna().copy()['nino34_mean'], model='multiplicable', period=12):

    from statsmodels.tsa.seasonal import seasonal_decompose
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go

    # descomposicion
    result = seasonal_decompose(data,
                                model=model,
                                period=period)

    # tipo de modelo
    if model=='multiplicable':
        modelo = 'Multiplicativo'
    elif model=='additive':
        modelo = 'Aditivo'
    else :
        modelo = ''

    # Creando figura
    fig = make_subplots(rows=4,cols=1,
                        column_titles=['SST promedio','Tendencia','Componente Estacional','Residuos']
                       )

    # Observado
    fig.append_trace(go.Scatter(x=result.observed.index, y=result.observed,
                                mode='lines',name='Tempreatura promedio observada NIÑO 3.4',
                                line=dict(color='#C10101', width=2)),
                     row=1, col=1)
    # Tendencia
    fig.append_trace(go.Scatter(x=result.trend.index, y=result.trend,
                                mode='lines',name='NIÑO 3.4',
                                line=dict(color='#FF0303', width=2)),
                     row=2, col=1)
    # Estacional
    fig.append_trace(go.Scatter(x=result.seasonal.index, y=result.seasonal,
                                mode='lines',name='NIÑO 3.4',
                                line=dict(color='#FF4242', width=2)),
                     row=3, col=1)
    # Residuales
    fig.append_trace(go.Scatter(x=result.resid.index, y=result.resid,
                                mode='lines',name='NIÑO 3.4',
                                line=dict(color='#FE5C5C', width=2)),
                     row=4, col=1)

    # Ejes
    fig['layout']['xaxis']['title']=''
    fig['layout']['xaxis2']['title']=''
    fig['layout']['xaxis3']['title']=''
    fig['layout']['xaxis4']['title']='Meses'


    fig['layout']['yaxis']['title']='Temperatura (°C)'
    fig['layout']['yaxis2']['title']='Tendencia'
    fig['layout']['yaxis3']['title']='Estacional'
    fig['layout']['yaxis4']['title']='Residuos'

    # Formato
    fig.update_xaxes(tickformat="%Y/%m",showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                     ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
    fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                     ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
    fig.update_layout(height=800,
                      width=1500,
                      template='plotly_white',
                      showlegend = False,
                      title_text="""
                                 Descomposición de la serie SST promedio región Niño 3.4
                                 <br><sup>Modelo {modelo}</sup>
                                 """.format(modelo=modelo),
                      xaxis_range=['1949-01-01','2023-04-01']
                     )
    return fig

In [20]:
# Modelo Aditivo
fig_aditivo = seasonal_decompose_plot(data=data_pandas[['nino34_mean']].dropna().copy(),
                                        model='additive',
                                        period=12)
fig_aditivo.write_image('./figure/nino34/descom_aditivo.png')
fig_aditivo.show()

In [21]:
# Modelo Multiplicativo
fig_multiplicativo = seasonal_decompose_plot(data=data_pandas[['nino34_mean']].dropna().copy(),
                                            model='multiplicable',
                                            period=12)
                                            
fig_multiplicativo.write_image('./figure/nino34/descom_multiplicativo.png')
fig_multiplicativo.show()

### Comportamiento climatológico

In [22]:
# Funcion para identificar las anomalias climaticas
def anomalia_periodo(year = 2020):

    value = list(filter(
                        lambda x: x[1] <= year and x[2] >= year,
                        [
                        [1,1950,1955,1936,1965],
                        [2,1956,1960,1941,1970],
                        [3,1961,1965,1946,1975],
                        [4,1966,1970,1951,1980],
                        [5,1971,1975,1956,1985],
                        [6,1976,1980,1961,1990],
                        [7,1981,1985,1966,1995],
                        [8,1986,1990,1971,2000],
                        [9,1991,1995,1976,2005],
                        [10,1996,2000,1981,2010],
                        [11,2001,2005,1986,2015],
                        [12,2006,2010,1991,2020],
                        [13,2011,2015,1996,2025],
                        [14,2016,2020,2001,2030],
                        [15,2021,2025,2006,2035],
                        [16,2026,2030,2011,2040],
                        [17,2031,2035,2016,2045],
                        [18,2036,2040,2021,2050],
                        [19,2041,2045,2026,2055],
                        [20,2046,2050,2031,2060],
                        [21,2051,2055,2036,2065],
                        ]
                        ))
    if value==[]:
        return 0
    else:
        return value[0][0]


def periodo_anomalias_climaticas(data_pandas):

    data_pandas['anomalia_periodo'] = data_pandas['year'].apply(anomalia_periodo)

    # determinando climatologica
    pd_climatologica = data_pandas.groupby(['month','anomalia_periodo'],as_index=False).agg({'climatologica':'mean'})

    # periodo de anomalias
    pd_periodo_anomalias = pd.DataFrame(
                                        [
                                    [1,1950,1955,1936,1965],
                                    [2,1956,1960,1941,1970],
                                    [3,1961,1965,1946,1975],
                                    [4,1966,1970,1951,1980],
                                    [5,1971,1975,1956,1985],
                                    [6,1976,1980,1961,1990],
                                    [7,1981,1985,1966,1995],
                                    [8,1986,1990,1971,2000],
                                    [9,1991,1995,1976,2005],
                                    [10,1996,2000,1981,2010],
                                    [11,2001,2005,1986,2015],
                                    [12,2006,2010,1991,2020],
                                    [13,2011,2015,1996,2025],
                                    [14,2016,2020,2001,2030],
                                    [15,2021,2025,2006,2035],
                                    [16,2026,2030,2011,2040],
                                    [17,2031,2035,2016,2045],
                                    [18,2036,2040,2021,2050],
                                    [19,2041,2045,2026,2055],
                                    [20,2046,2050,2031,2060],
                                    [21,2051,2055,2036,2065],
                                    ],
                            columns=['anomalia_periodo','date_init','date_end','range_init','range_end']

                        )


    pd_climatologia_final = pd.merge(pd_periodo_anomalias, pd_climatologica,on=['anomalia_periodo'],how='inner')
    return pd_climatologia_final[['date_init','date_end','anomalia_periodo','month','climatologica','range_init','range_end']]


# Data de anomalias climaticas
pd_perioodo_anomalias = periodo_anomalias_climaticas(data_pandas)
pd_perioodo_anomalias.head()

,date_init,date_end,anomalia_periodo,month,climatologica,range_init,range_end
0,1950,1955,1,1,26.18,1936,1965
1,1950,1955,1,2,26.39,1936,1965
2,1950,1955,1,3,26.95,1936,1965
3,1950,1955,1,4,27.39,1936,1965
4,1950,1955,1,5,27.56,1936,1965


In [23]:
# Data temperatura promedio
pd_temperatura = pd.merge(data_pandas[['year','month','mes','mes_year','nino34_mean','periodo','anomalia_periodo']],
                         pd_perioodo_anomalias,
                         on=['month','anomalia_periodo'],
                         how='left')
pd_temperatura['base'] = pd_temperatura\
                            .apply(lambda x: str(int(x.range_init))+'-'+str(int(x.range_end)),1 )

pd_temperatura.head()

,year,month,mes,mes_year,nino34_mean,periodo,anomalia_periodo,date_init,date_end,climatologica,range_init,range_end,base
0,2022,6,junio,junio-2022,26.96,2022-06-01,15,2021,2025,27.73,2006,2035,2006-2035
1,2022,5,mayo,mayo-2022,26.82,2022-05-01,15,2021,2025,27.94,2006,2035,2006-2035
2,2022,4,abril,abril-2022,26.71,2022-04-01,15,2021,2025,27.83,2006,2035,2006-2035
3,2022,3,marzo,marzo-2022,26.32,2022-03-01,15,2021,2025,27.29,2006,2035,2006-2035
4,2022,2,febrero,febrero-2022,25.87,2022-02-01,15,2021,2025,26.76,2006,2035,2006-2035


In [24]:
# Data temperatura periodo base
pd_normalidad = pd_perioodo_anomalias.copy()
pd_normalidad['base'] = pd_normalidad\
                            .apply(lambda x: str(int(x.range_init))+'-'+str(int(x.range_end)),1 )
#pd_normalidad['mes'] = pd_normalidad['month'].apply(mese_text)
pd_normalidad['mes'] =  pd.to_datetime(pd_normalidad['month'],format='%m',errors='ignore')
pd_normalidad['mes'] =  pd_normalidad['mes'].dt.strftime('%B')
pd_normalidad.head()

,date_init,date_end,anomalia_periodo,month,climatologica,range_init,range_end,base,mes
0,1950,1955,1,1,26.18,1936,1965,1936-1965,enero
1,1950,1955,1,2,26.39,1936,1965,1936-1965,febrero
2,1950,1955,1,3,26.95,1936,1965,1936-1965,marzo
3,1950,1955,1,4,27.39,1936,1965,1936-1965,abril
4,1950,1955,1,5,27.56,1936,1965,1936-1965,mayo


In [25]:
import plotly.graph_objects as go
from plotly.graph_objects import Layout

fig = go.Figure(layout=Layout(plot_bgcolor='rgba(0,0,0,0)'))
periodos = pd_normalidad.base.unique().tolist()
color = [
        '#FB5CFF','#F700FF','#910095',
        '#7E8EFA','#5B67BD', '#0022FF',
        '#6ADDFF','#00C5FF', '#0092BD',
        #'#6FFE76','#00FF0C','#00A808',
        '#F8FF20','#DFF100','#AEBB02',
        '#FFC162','#FFB900','#DA9E00']

for base in periodos:
    
    data_base = pd_normalidad\
                    .query("base=='{base}'".format(base=base))\
                    .sort_values(['date_init','month'],ascending=[True,True])\
                    .copy()
    data_temperatura = pd_temperatura\
                        .query("base=='{base}'".format(base=base))\
                        .sort_values(['date_init','month'],ascending=[True,True])\
                        .copy()
    # colores
    col = color[periodos.index(base)]
    
    fig.add_trace(go.Scatter(x=data_base.mes, y=data_base['climatologica'],
                             mode='lines+markers',
                             marker_symbol='x-thin',
                             marker_line_width=2,
                             marker_size=5,
                             marker_line_color=col,
                             marker_color=col,
                             name=base,
                             line=dict(color=col, width=2), #0.5*(periodos.index(base)+1)/2),
                             showlegend=True))
    fig.add_trace(go.Box(x=data_temperatura.mes, y=data_temperatura['nino34_mean'],
                         opacity=0.1,
                         name=base,line=dict(color=col),
                        showlegend=False
                 ))



fig.update_xaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)



#fig.update_traces(textfont_size=18)
fig.update_layout(title="""
                        SST promedio en la región Niño 3.4
                        <br><sup>Periodos base de 30 años</sup>
                        """,
                  xaxis_title='Mes',
                  yaxis_title='SST promedio (°C)',
                  legend_title_text='Periodo base',
                  legend_title = dict( font = dict(size = 20)),
                  legend=dict(y=0.5,
                              traceorder='reversed',
                              font_size=11),

                   uniformtext_minsize=8,
                   uniformtext_mode='hide',
                   height=800,
                   width=1500,
                   font = dict(size = 20)
                 )

fig.write_image('./figure/nino34/climatologia.png')
fig.show()

In [26]:
# Calculando del ONI
pd_oni_test = pd_temperatura.copy()
pd_oni_test['anomalias'] = pd_oni_test['nino34_mean'] - pd_oni_test['climatologica']

pd_oni_test['oni'] =  pd_oni_test.anomalias.rolling(min_periods=1, window=3, center=True).mean()
pd_oni_test['periodo'] =  pd.to_datetime(pd_oni_test['periodo'],format='%Y-%m',errors='ignore')

pd_oni_test = pd_oni_test.sort_values('periodo',ascending=True)
pd_oni_test.index = pd.to_datetime(pd_oni_test.periodo)
pd_oni_test.head()

,year,month,mes,mes_year,nino34_mean,periodo,anomalia_periodo,date_init,date_end,climatologica,range_init,range_end,base,anomalias,oni
periodo,,,,,,,,,,,,,,,
1950-01-01,1950,1,enero,enero-1950,24.56,1950-01-01,1,1950,1955,26.18,1936,1965,1936-1965,-1.62,-1.470000
1950-02-01,1950,2,febrero,febrero-1950,25.07,1950-02-01,1,1950,1955,26.39,1936,1965,1936-1965,-1.32,-1.336667
1950-03-01,1950,3,marzo,marzo-1950,25.88,1950-03-01,1,1950,1955,26.95,1936,1965,1936-1965,-1.07,-1.163333
1950-04-01,1950,4,abril,abril-1950,26.29,1950-04-01,1,1950,1955,27.39,1936,1965,1936-1965,-1.10,-1.180000
1950-05-01,1950,5,mayo,mayo-1950,26.19,1950-05-01,1,1950,1955,27.56,1936,1965,1936-1965,-1.37,-1.070000


In [27]:
import plotly.graph_objects as go
from plotly.graph_objects import Layout

data_fig = pd_oni_test.copy()
data_fig['color'] = data_fig['oni'].apply(lambda x: 0 if x<0 else 1)


fig = go.Figure(layout=Layout(plot_bgcolor='rgba(0,0,0,0)'))
fig.add_trace(go.Scatter(x=data_fig.index.tolist(), y=len(data_fig.index.tolist())*[0],
                         mode='lines',name='NIÑO3.4 NARX entrenamiento',
                         line=dict(color='#B0ACAC', width=2),
                         fill = 'tozeroy',
                         fillcolor = '#F5FF8D',
                        showlegend=False))#,fill='tozeroy'))

fig.add_trace(go.Scatter(x=data_fig.index, 
                         y=data_fig.oni,
                         mode='lines+markers',
                         marker_symbol='x-thin',
                         marker_line_width=2,
                         marker_size=3,
                         marker_line_color='#003CAF',
                         marker_color='#003CAF',
                         name='NIÑO3.4 NARX entrenamiento',
                         line=dict(color='#0057FF', width=3),
                         fill = 'tonexty',
                         fillcolor = '#8FB5FE',
                         showlegend=False
                        ))

fig.add_annotation(x=pd.Timestamp('2026-04-01'), y=0.5+0.2,text="débil",showarrow=False,yshift=10,font=dict(color='#FF6C6C') )
fig.add_hline(y=0.5, line_width=0.75, line_dash="dash", line_color="#FF6C6C")

fig.add_annotation(x=pd.Timestamp('2026-04-01'), y=1+0.2,text="moderado",showarrow=False,yshift=10,font=dict(color='#FF3F3F') )
fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="#FF3F3F")

fig.add_annotation(x=pd.Timestamp('2026-04-01'), y=1.5+0.2,text="fuerte",showarrow=False,yshift=10,font=dict(color='#FF0000') )
fig.add_hline(y=1.5, line_width=1.25, line_dash="dash", line_color="#FF0000")

fig.add_annotation(x=pd.Timestamp('2026-04-01'), y=2+0.2,text="muy fuerte",showarrow=False,yshift=10,font=dict(color='#D70000') )
fig.add_hline(y=2, line_width=1.50, line_dash="dash", line_color="#D70000")


fig.add_hline(y=2.5, line_width=1.75, line_dash="dash", line_color="#AD0000")


fig.add_annotation(x=pd.Timestamp('2026-04-01'), y=-0.5-0.35,text="débil",showarrow=False,yshift=10,font=dict(color='#69A6FF') )
fig.add_hline(y=-0.5, line_width=0.75, line_dash="dash", line_color="#69A6FF")

fig.add_annotation(x=pd.Timestamp('2026-04-01'), y=-1-0.35,text="moderado",showarrow=False,yshift=10,font=dict(color='#6979FF') )
fig.add_hline(y=-1, line_width=1, line_dash="dash", line_color="#6979FF")

fig.add_annotation(x=pd.Timestamp('2026-04-01'), y=-1.5-0.35,text="fuerte",showarrow=False,yshift=10,font=dict(color='#3F53FF') )
fig.add_hline(y=-1.5, line_width=1.25, line_dash="dash", line_color="#3F53FF")

fig.add_annotation(x=pd.Timestamp('2026-04-01'), y=-2-0.35,text="muy fuerte",showarrow=False,yshift=10,font=dict(color='#001BFF') )
fig.add_hline(y=-2, line_width=1.5, line_dash="dash", line_color="#001BFF")

fig.add_hline(y=-2.5, line_width=1.75, line_dash="dash", line_color="#00059A")


fig.add_annotation(x=pd.Timestamp('1960-01-01'), y=2+0.2,text="El Niño",showarrow=False,yshift=15,font=dict(color='#D70000') )
fig.add_annotation(x=pd.Timestamp('1960-01-01'), y=-2-0.35,text="La Niña",showarrow=False,yshift=15,font=dict(color='#001BFF') )


fig.update_xaxes(tickformat="%Y/%m",showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)

fig.update_layout(title="""
                        Índice Niño Oceánico (ONI) periodo {date_init} al {date_fin}
                        <br><sup>Promedio de 3-meses para las anomalías SST en la región Niño 3.4 (variación periodos base de 30-años)
                        </sup>
                        """.format(date_init=str(data_fig.index.min().strftime('%Y-%m-%d')),
                                   date_fin=str(data_fig.index.max().strftime('%Y-%m-%d')) ),
                  xaxis_title='Mes',
                  yaxis_title='Promedio 3-Meses anomalías SST (°C)',
#                   legend_title_text='Serie',
#                   legend_title = dict( font = dict(size = 30)),
#                   legend=dict(y=0.5,
#                               #traceorder='reversed',
#                               font_size=22),

                   uniformtext_minsize=8,
                   uniformtext_mode='hide',
                   height=800,
                   width=1500,
                   font = dict(size = 22),
                   xaxis_range=['1949-01-01','2030-04-01']
                 )


fig.write_image('./figure/nino34/oni.png')
fig.show()

### Ajustando modelos

In [48]:
# Data modelo
data_model = data_pandas[['nino34_mean']].dropna().copy()

In [49]:
import numpy as np
import pandas as pd

#https://scikit-learn.org/stable/modules/neural_networks_supervised.html

class NARX_NINO34():
    
    def __init__(self,pd_model, rango_prediccion = 24, auto_order = 12*20, 
                     exog_order = [12*20],exog_delay = [1]):
        
        self.pd_model = pd_model
        self.rango_prediccion = rango_prediccion
        self.auto_order = auto_order
        self.exog_order = exog_order
        self.exog_delay = exog_delay
    
    def split_data(self,y='nino34_mean',exogena='nino34_mean'):
        
        self.y = y
        self.exogena = exogena

        self.t_y = 't_' + self.y
        self.t_exogena = 't_' + self.exogena

        from sklearn.preprocessing import MinMaxScaler
        self.minimax_y = MinMaxScaler().fit( self.pd_model[[self.y]] )
        self.minimax_exogena = MinMaxScaler().fit( self.pd_model[[self.exogena]] )

        self.pd_model[self.t_y] = self.minimax_y.transform(self.pd_model[[self.y]])
        self.pd_model[self.t_exogena] = self.minimax_exogena.transform(self.pd_model[[self.exogena]])

        # dividiendo los datos
        self.pd_training = self.pd_model.iloc[:-self.rango_prediccion]
        self.pd_test = self.pd_model.iloc[self.pd_training.shape[0]:]

        self.x_training = self.pd_training[[self.t_exogena]].to_numpy().astype(np.float)
        self.y_training = self.pd_training[[self.t_y]].to_numpy().astype(np.float)
        self.y_training = self.y_training.reshape(self.y_training.shape[0],)

        self.x_test = self.pd_test[[self.t_exogena]].to_numpy().astype(np.float)
        self.y_test = self.pd_test[[self.t_y]].to_numpy().astype(np.float)
        self.y_test = self.y_test.reshape(self.y_test.shape[0],)
        
    def fit(self,para_grid):
        
        self.para_grid = para_grid
        
        from fireTS.models import NARX
        from sklearn.neural_network import MLPRegressor
        
        self.clase_regresor = MLPRegressor(hidden_layer_sizes=(44), 
                                        activation='tanh',
                                        solver='adam',
                                        alpha=0.0001,
                                        batch_size='auto',
                                        learning_rate='adaptive',
                                        learning_rate_init=0.001,
                                        power_t=0.5,
                                        max_iter=10000,
                                        shuffle=True,
                                        random_state=None,
                                        tol=0.0001,
                                        verbose=False,
                                        warm_start=False,
                                        momentum=0.9,
                                        nesterovs_momentum=True,
                                        early_stopping=False,
                                        validation_fraction=0.1,
                                        beta_1=0.9,
                                        beta_2=0.999,
                                        epsilon=1e-08,
                                        n_iter_no_change=10,
                                        max_fun=15000)
        
        # NARX model
        self.model = NARX(self.clase_regresor,
                     auto_order=self.auto_order,
                     exog_order=self.exog_order,
                     exog_delay=self.exog_delay)

        self.model.grid_search(self.x_training, self.y_training, self.para_grid)#, verbose=0)
        self.model.fit(self.x_training, self.y_training)

        # Estudiando el ajuste
        self.pd_ajuste = self.pd_training.copy()
        self.pd_ajuste['t_y_predict'] = self.model.predict(self.x_training, self.y_training, step=1)
        self.pd_ajuste['y_predict'] = self.minimax_y.inverse_transform(self.pd_ajuste[['t_y_predict']])
        
    def predict(self):
        
        # auto prediccion
        for i in range(1, self.rango_prediccion + 1):
            if i == 1:
                self.y_forecast = self.model.forecast(self.x_training, self.y_training, X_future = None, step=1)
                self.X_future=self.y_forecast.reshape(-1, 1)
            else:
                self.y_forecast = self.model.forecast(self.x_training, self.y_training, X_future = self.X_future, step=i)
                self.X_future = self.y_forecast.reshape(-1, 1)

        # Estudiando el ajuste
        self.pd_validacion = self.pd_test.copy()
        self.pd_validacion['t_y_predict'] = self.y_forecast
        self.pd_validacion['y_predict'] = self.minimax_y.inverse_transform(self.pd_validacion[['t_y_predict']])

    def forecast(self,step=None):
        if step is None:
            step=self.rango_prediccion
        
        y = self.pd_model[[self.t_y]].to_numpy().astype(np.float)
        y = y.reshape(y.shape[0],)
        X = self.pd_model[[self.t_exogena]].to_numpy().astype(np.float)
        
        # auto prediccion
        for i in range(1,step+1):
            if i == 1:
                y_forecast = self.model.forecast(X, y, X_future = None, step=1)
                X_future=y_forecast.reshape(-1, 1)
            else:
                y_forecast = self.model.forecast(X, y, X_future = X_future, step=i)
                X_future = y_forecast.reshape(-1, 1)
                
                
        pd_forecast = pd.DataFrame(y_forecast,columns=['t_y_predict'])
        pd_forecast['y_predict'] = self.minimax_y.inverse_transform( pd_forecast[['t_y_predict']] )
        date_range = pd.to_datetime(pd.date_range(
                                        start=self.pd_model.index.max(),
                                        freq='M',
                                        periods=step+1)\
                                        .strftime("%Y%m"),
                                    format='%Y%m' )[1:]
        pd_forecast['periodo'] = date_range
        pd_forecast = pd_forecast.set_index('periodo')
        pd_forecast.index = pd.to_datetime(pd_forecast.index)
        return pd_forecast
        
        
    def metricas(self):
        
        pd_observado = self.pd_ajuste.dropna().copy()

        mobservado = NARX_NINO34.metrics(hidden_layer_sizes=self.model.base_estimator.hidden_layer_sizes,
                                         observado=pd_observado[self.y],
                                         prediccion=pd_observado.y_predict)

        pd_mobservado = pd.DataFrame(mobservado)
        pd_mobservado = pd_mobservado.rename(columns={"hidden_layer_sizes": "entrenemiento_hidden_layer_sizes", 
                                                      "map": "entrenemiento_map",
                                                      "mae": "entrenemiento_mae",
                                                      "rmse": "entrenemiento_rmse",
                                                      "r2": "entrenemiento_r2"
                                                     })



        mvalidacion = NARX_NINO34.metrics(hidden_layer_sizes=self.model.base_estimator.hidden_layer_sizes,
                                         observado=self.pd_validacion[self.y],
                                         prediccion=self.pd_validacion.y_predict)

        pd_mvalidacion = pd.DataFrame(mvalidacion)
        pd_mvalidacion = pd_mvalidacion.rename(columns={"hidden_layer_sizes": "validacion_hidden_layer_sizes", 
                                                      "map": "validacion_map",
                                                      "mae": "validacion_mae",
                                                      "rmse": "validacion_rmse",
                                                      "r2": "validacion_r2"
                                                     })

        self.metricas = pd.concat([pd_mobservado,pd_mvalidacion],axis=1)
        self.metricas['rango_prediccion'] = self.rango_prediccion
        self.metricas['auto_order'] = self.auto_order 
        self.metricas['exog_order'] = self.exog_order 
        self.metricas['exog_delay'] = self.exog_delay 

        self.metricas=self.metricas[['entrenemiento_hidden_layer_sizes',
                                     'rango_prediccion',
                                     'auto_order',
                                     'exog_order',
                                     'exog_delay',
                                     'entrenemiento_map',
                                     'entrenemiento_mae',
                                     'entrenemiento_rmse',
                                     'entrenemiento_r2',
                                     'validacion_map',
                                     'validacion_mae',
                                     'validacion_rmse',
                                     'validacion_r2']]
        
    
    @staticmethod
    def metrics(hidden_layer_sizes,observado,prediccion):
        
        from sklearn.metrics import (mean_absolute_percentage_error,mean_absolute_error,mean_squared_error,r2_score)

        return {'hidden_layer_sizes':[hidden_layer_sizes],
              'map':[mean_absolute_percentage_error(observado, prediccion)],
              'mae':[mean_absolute_error(observado, prediccion)],
              'rmse':[mean_squared_error(observado, prediccion,squared=False)],
              'r2': [r2_score(observado, prediccion,multioutput='variance_weighted')],
              }
    
    
    def graf(self):
        
        import plotly.graph_objects as go

        data_figure_ajuste = self.pd_ajuste.dropna()
        data_figure_validacion = self.pd_validacion.dropna()
        data_figure_pronostico = self.forecast()


        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data_figure_ajuste.index, y=data_figure_ajuste['y_predict'],
                                 mode='lines+markers',name='NIÑO3.4 NARX entrenamiento',
                                 line=dict(color='#AE04B9', width=2)))

        fig.add_trace(go.Scatter(x=data_figure_ajuste.index, y=data_figure_ajuste[self.y],
                                 mode='lines',name='NIÑO3.4 observado entrenamiento',
                                 line=dict(color='#B90404', width=2)))

        fig.add_trace(go.Scatter(x=data_figure_validacion.index, y=data_figure_validacion['y_predict'],
                            mode='lines+markers',name='NIÑO3.4 NARX validación',
                            line=dict(color='#F000FF', width=2, dash='dash')))

        fig.add_trace(go.Scatter(x=data_figure_validacion.index, y=data_figure_validacion[self.y],
                            mode='lines',name='NIÑO3.4 observado validación',
                            line=dict(color='#FF0000', width=2)))

        fig.add_trace(go.Scatter(x=data_figure_pronostico.index, y=data_figure_pronostico['y_predict'],
                            mode='lines+markers',name='NIÑO3.4 NARX pronóstico',
                            line=dict(color='#E466FF', width=2,dash='dash')))
        
        fig.add_vline(x=data_figure_ajuste.index.max(), line_width=3, line_dash="dash", line_color="#580606")
        fig.add_vline(x=data_figure_pronostico.index.min(), line_width=3, line_dash="dash", line_color="#580606")
        fig.update_layout(title="""
                            Temperatura promedio en la región NIÑO3.4 
                            <br><sup>Pronóstico para el periodo {date_init} al {date_fin}</sup>
                            """.format(date_init=str(data_figure_pronostico.index.min().strftime('%Y-%m-%d')),
                                       date_fin=str(data_figure_pronostico.index.max().strftime('%Y-%m-%d')) ),
                          xaxis_title='Mes',
                          yaxis_title='Temperatura (C)',
                          legend_title_text='Serie',
                          legend_title = dict( font = dict(size = 30)),
                          legend=dict(y=0.5,
                                      #traceorder='reversed',
                                      font_size=22),

                           uniformtext_minsize=8,
                           uniformtext_mode='hide',
                           height=800,
                           width=1500,
                           font = dict(size = 22)
                         )

        fig.show()

In [50]:
class TRAINING_NINO34_MODELS():
  
    def __init__(self,pd_modelo,l_rango_prediccion,l_auto_order,l_exog_order,l_exog_delay,l_exogena,l_y):
        self.pd_modelo=pd_modelo 
        self.l_rango_prediccion=l_rango_prediccion
        self.l_auto_order=l_auto_order
        self.l_exog_order=l_exog_order
        self.l_exog_delay=l_exog_delay
        self.l_exogena=l_exogena 
        self.l_y=l_y
    
    def training(self):
        
        #---------
        def fit_model(pd_modelo,rango_prediccion,auto_order,exog_order,exog_delay,exogena,y):
            
            # ajustando modelo
            modelo_nino34 = NARX_NINO34(pd_model=pd_modelo.copy(),
                                        rango_prediccion = rango_prediccion,
                                        auto_order = auto_order, 
                                        exog_order = exog_order,
                                        exog_delay = exog_delay)

            neuronas = modelo_nino34.auto_order+modelo_nino34.exog_order[0]
            hiden = [
#                     (int(neuronas),
#                       int(neuronas/2),
#                       int(neuronas/4)
#                      ),
                     (int(neuronas),
                      2*int(neuronas),
                      int(neuronas)
                     ),
#                      (int(neuronas),
#                       2*int(neuronas),
#                       int(neuronas),
#                       int(neuronas/2)
#                      )
                    ]
            # busqueda
            para_grid = {'hidden_layer_sizes': hiden}
            modelo_nino34.split_data(y=y,exogena=exogena)
            modelo_nino34.fit(para_grid=para_grid)
            modelo_nino34.predict()
            modelo_nino34.metricas()
            
            return modelo_nino34
        #---------
        
        self.list_models = list(map(lambda  rango_prediccion,auto_order,exog_order,exog_delay,exogena,y : fit_model(pd_modelo=self.pd_modelo.copy(),rango_prediccion=rango_prediccion,auto_order=auto_order,exog_order=exog_order,exog_delay=exog_delay,exogena=exogena,y=y),
                                self.l_rango_prediccion,
                                self.l_auto_order,
                                self.l_exog_order,
                                self.l_exog_delay,
                                self.l_exogena,
                                self.l_y
                                ))
        
        self.summary = pd.concat(list(map(lambda x: x.metricas, self.list_models)))

In [51]:
# PARAMETROS
l_rango_prediccion= [6,12,18,24,30,36] +\
                    [6,12,18,24,30,36] +\
                    [6,12,18,24,30,36] +\
                    [6,12,18,24,30,36] +\
                    [6,12,18,24,30,36]
l_auto_order= 6*[12*5] +\
              6*[12*10] +\
              6*[12*20] +\
              6*[12*30] +\
              6*[12*40]
l_exog_order= 6*[[12*5]] +\
              6*[[12*10]] +\
              6*[[12*20]] +\
              6*[[12*30]] +\
              6*[[12*40]] 
l_exog_delay= 6*[[1]] +\
              6*[[1]] +\
              6*[[1]] +\
              6*[[1]] +\
              6*[[1]] 
l_exogena= len(l_auto_order)*['nino34_mean']
l_y= len(l_auto_order)*['nino34_mean'] 

# ENTRENANDO MODELOS
np.random.seed(0)

modelos = TRAINING_NINO34_MODELS(pd_modelo=data_model[['nino34_mean']].dropna().copy(),
                                 l_rango_prediccion=l_rango_prediccion,
                                 l_auto_order=l_auto_order,
                                 l_exog_order=l_exog_order,
                                 l_exog_delay=l_exog_delay,
                                 l_exogena=l_exogena,
                                 l_y=l_y)
modelos.training()
modelos.summary

/tmp/ipykernel_69528/741000829.py:36: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/tmp/ipykernel_69528/741000829.py:37: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

/tmp/ipykernel_69528/741000829.py:40: DeprecationWarning:

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing t

In [23]:
# Mejor modelo de temperatura
best_model = list(filter(lambda x: x.rango_prediccion==12 and x.auto_order==60, modelo_nino34.list_models))[0]

In [24]:
import plotly.graph_objects as go
from plotly.graph_objects import Layout

data_figure_ajuste = best_model.pd_ajuste.dropna()
data_figure_validacion = best_model.pd_validacion.dropna()
data_figure_pronostico = best_model.forecast()



fig = go.Figure(layout=Layout(plot_bgcolor='rgba(0,0,0,0)'))



fig.add_annotation(x=pd.Timestamp('2019-01-01'), y=29,
            text="Entrenamiento",
            showarrow=False,
            yshift=10)

fig.add_trace(go.Scatter(x=data_figure_ajuste.index, y=data_figure_ajuste['y_predict'],
                         mode='lines+markers',name='Pronóstico entrenamiento',
                         marker_symbol='x-thin',
                         marker_line_width=3,
                         marker_size=3,
                         marker_line_color='#000000',
                         marker_color='#000000',
                         line=dict(color='#FF7203', width=2)))

fig.add_trace(go.Scatter(x=data_figure_ajuste.index, y=data_figure_ajuste[best_model.y],
                         mode='lines+markers',name='SSTT entrenamiento',
                         marker_symbol='x-thin',
                         marker_line_width=3,
                         marker_size=3,
                         marker_line_color='#000000',
                         marker_color='#000000',
                         line=dict(color='#C10101', width=2)))




fig.add_annotation(x=pd.Timestamp('2021-07-01'), y=29,
            text="Validación",
            showarrow=False,
            yshift=10)
fig.add_trace(go.Scatter(x=data_figure_validacion.index, y=data_figure_validacion['y_predict'],
                    mode='lines+markers',name='Pronóstico validación',                       
                         marker_symbol='square',
                         marker_line_width=2,
                         marker_size=3,
                         marker_line_color='#030BFF',
                         marker_color='#030BFF', 
                         line=dict(color='#FF7203', width=2)
                        ))

fig.add_trace(go.Scatter(x=data_figure_validacion.index, y=data_figure_validacion[best_model.y],
                    mode='lines+markers',name='SSTT validación',
                    marker_symbol='square',
                    marker_line_width=2,
                    marker_size=3,
                    marker_line_color='#030BFF',
                    marker_color='#030BFF', 
                    line=dict(color='#C10101', width=2)))



fig.add_annotation(x=pd.Timestamp('2022-09-01'), y=29,
            text="Pronóstico",
            showarrow=False,
            yshift=10)
fig.add_trace(go.Scatter(x=data_figure_pronostico.index, y=data_figure_pronostico['y_predict'],
                         text=data_figure_pronostico['y_predict'].apply(lambda x: str(round(x,2)) ),
                         textposition="top right",
                         marker_symbol='star',
                         marker_line_width=3,
                         marker_size=3,
                         marker_line_color='#EA9800',
                         marker_color='#EA9800',
                         mode='lines+markers+text',name='Pronóstico SSTT',
                         line=dict(color='#FF7203', width=2,dash='dash')))

fig.add_vline(x=data_figure_ajuste.index.max(), line_width=3, line_dash="dash", line_color="#580606")
fig.add_vline(x=data_figure_validacion.index.max(), line_width=3, line_dash="dash", line_color="#580606")


fig.update_xaxes(tickformat="%Y/%m",showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)

fig.update_traces(textfont_size=11)
fig.update_layout(title="""
                    SST promedio en la región NIÑO 3.4 
                    <br><sup>Pronóstico para el periodo {date_init} al {date_fin}</sup>
                    """.format(date_init=str(data_figure_pronostico.index.min().strftime('%Y-%m-%d')),
                               date_fin=str(data_figure_pronostico.index.max().strftime('%Y-%m-%d')) ),
                  xaxis_title='Mes',
                  yaxis_title='Temperatura (°C)',
                  legend_title_text='Serie',
                  legend_title = dict( font = dict(size = 25)),
                  legend=dict(y=0.5,
                              #traceorder='reversed',
                              font_size=22),

                   uniformtext_minsize=8,
                   uniformtext_mode='hide',
                   height=800,
                   width=1500,
                   font = dict(size = 22),
                   xaxis_range=['2016-11-01','2023-04-01'],
                 )

fig.show()

In [25]:
# Funcion para identificar las anomalias climaticas
def anomalia_periodo(year = 2020):

    value = list(filter(
                        lambda x: x[1] <= year and x[2] >= year,
                        [
                        [1,1950,1955,1936,1965],
                        [2,1956,1960,1941,1970],
                        [3,1961,1965,1946,1975],
                        [4,1966,1970,1951,1980],
                        [5,1971,1975,1956,1985],
                        [6,1976,1980,1961,1990],
                        [7,1981,1985,1966,1995],
                        [8,1986,1990,1971,2000],
                        [9,1991,1995,1976,2005],
                        [10,1996,2000,1981,2010],
                        [11,2001,2005,1986,2015],
                        [12,2006,2010,1991,2020],
                        [13,2011,2015,1996,2025],
                        [14,2016,2020,2001,2030],
                        [15,2021,2025,2006,2035],
                        [16,2026,2030,2011,2040],
                        [17,2031,2035,2016,2045],
                        [18,2036,2040,2021,2050],
                        [19,2041,2045,2026,2055],
                        [20,2046,2050,2031,2060],
                        [21,2051,2055,2036,2065],
                        ]
                        ))
    if value==[]:
        return 0
    else:
        return value[0][0]


def periodo_anomalias_climaticas(data_pandas):

    data_pandas[['anomalia_periodo']] = data_pandas['year'].apply(anomalia_periodo)

    # determinando climatologica
    pd_climatologica = data_pandas.groupby(['month','anomalia_periodo'],as_index=False).agg({'climatologica':'mean'})

    # periodo de anomalias
    pd_periodo_anomalias = pd.DataFrame(
                                        [
                                    [1,1950,1955,1936,1965],
                                    [2,1956,1960,1941,1970],
                                    [3,1961,1965,1946,1975],
                                    [4,1966,1970,1951,1980],
                                    [5,1971,1975,1956,1985],
                                    [6,1976,1980,1961,1990],
                                    [7,1981,1985,1966,1995],
                                    [8,1986,1990,1971,2000],
                                    [9,1991,1995,1976,2005],
                                    [10,1996,2000,1981,2010],
                                    [11,2001,2005,1986,2015],
                                    [12,2006,2010,1991,2020],
                                    [13,2011,2015,1996,2025],
                                    [14,2016,2020,2001,2030],
                                    [15,2021,2025,2006,2035],
                                    [16,2026,2030,2011,2040],
                                    [17,2031,2035,2016,2045],
                                    [18,2036,2040,2021,2050],
                                    [19,2041,2045,2026,2055],
                                    [20,2046,2050,2031,2060],
                                    [21,2051,2055,2036,2065],
                                    ],
                            columns=['anomalia_periodo','date_init','date_end','range_init','range_end']

                        )


    pd_climatologia_final = pd.merge(pd_periodo_anomalias,pd_climatologica,on=['anomalia_periodo'],how='inner')
    return pd_climatologia_final[['date_init','date_end','anomalia_periodo','month','climatologica','range_init','range_end']]


# Data de anomalias climaticas
pd_perioodo_anomalias = periodo_anomalias_climaticas(data_pandas.copy())
pd_perioodo_anomalias.head()

,date_init,date_end,anomalia_periodo,month,climatologica,range_init,range_end
0,1950,1955,1,1,26.18,1936,1965
1,1950,1955,1,2,26.39,1936,1965
2,1950,1955,1,3,26.95,1936,1965
3,1950,1955,1,4,27.39,1936,1965
4,1950,1955,1,5,27.56,1936,1965


In [26]:
# data de integracion del modelo
pd_ajuste = best_model.pd_ajuste[[best_model.y]].copy()
pd_ajuste['type']='ajuste'

pd_validacion = best_model.pd_validacion[[best_model.y]].copy()
pd_validacion['type']='validacion'

pd_pronostico = best_model.forecast()[['y_predict']].rename(columns={'y_predict':best_model.y}).copy()
pd_pronostico['type']='pronostico'

pd_integracion = pd.concat([
                            pd_ajuste,
                            pd_validacion,
                            pd_pronostico
                            ])

pd_integracion['year'] = pd_integracion.index.strftime('%Y').astype(int)
pd_integracion['month'] = pd_integracion.index.strftime('%m').astype(int)
pd_integracion['anomalia_periodo'] = pd_integracion['year'].apply(lambda x: anomalia_periodo(int(x)) ).astype(int)

pd_integracion[pd_integracion.year==2020]

,nino34_mean,type,year,month,anomalia_periodo
periodo,,,,,
2020-01-01,27.16,ajuste,2020,1,14
2020-02-01,27.13,ajuste,2020,2,14
2020-03-01,27.77,ajuste,2020,3,14
2020-04-01,28.19,ajuste,2020,4,14
2020-05-01,27.66,ajuste,2020,5,14
2020-06-01,27.39,ajuste,2020,6,14
2020-07-01,26.99,ajuste,2020,7,14
2020-08-01,26.27,ajuste,2020,8,14
2020-09-01,25.89,ajuste,2020,9,14


In [31]:
# incorporando anomalias al pronostico
pd_oni = pd.merge(pd_integracion,
                 pd_perioodo_anomalias,
                 on=['month','anomalia_periodo'],
                 how='left')[['nino34_mean','type','year','month','climatologica']]

pd_oni['anomalias'] = pd_oni['nino34_mean'] - pd_oni['climatologica']
pd_oni['oni'] =  pd_oni.anomalias.rolling(min_periods=1, window=3, center=True).mean()
pd_oni = pd_oni.dropna().copy()
pd_oni['periodo'] =  pd_oni.apply(lambda x:str(int(x.year))+'-'+str(int(x.month)),1)
pd_oni['periodo'] =  pd.to_datetime(pd_oni['periodo'],format='%Y-%m',errors='ignore')
pd_oni = pd_oni.sort_values('periodo',ascending=True)
pd_oni.index = pd.to_datetime(pd_oni.periodo)

pd_oni = pd_oni[pd_oni.index<pd_oni.index.max()].copy()

pd_oni.head()

,nino34_mean,type,year,month,climatologica,anomalias,oni,periodo
periodo,,,,,,,,
1950-01-01,24.56,ajuste,1950,1,26.18,-1.62,-1.470000,1950-01-01
1950-02-01,25.07,ajuste,1950,2,26.39,-1.32,-1.336667,1950-02-01
1950-03-01,25.88,ajuste,1950,3,26.95,-1.07,-1.163333,1950-03-01
1950-04-01,26.29,ajuste,1950,4,27.39,-1.10,-1.180000,1950-04-01
1950-05-01,26.19,ajuste,1950,5,27.56,-1.37,-1.070000,1950-05-01


In [32]:
import plotly.graph_objects as go
from plotly.graph_objects import Layout

data_fig = pd_oni.copy()
data_fig['color'] = data_fig['oni'].apply(lambda x: 0 if x<0 else 1)


fig = go.Figure(layout=Layout(plot_bgcolor='rgba(0,0,0,0)'))
fig.add_trace(go.Scatter(x=data_fig.index.tolist(), y=len(data_fig.index.tolist())*[0],
                         mode='lines',name='NIÑO3.4 NARX entrenamiento',
                         line=dict(color='#B0ACAC', width=2),
                         fill = 'tozeroy',
                         fillcolor = '#F5FF8D',
                        showlegend=False))#,fill='tozeroy'))

fig.add_trace(go.Scatter(x=data_fig.index, 
                         y=data_fig.oni,
                         mode='lines+markers',
                         marker_symbol='x-thin',
                         marker_line_width=2,
                         marker_size=3,
                         marker_line_color='#003CAF',
                         marker_color='#003CAF',
                         name='NIÑO3.4 NARX entrenamiento',
                         line=dict(color='#0057FF', width=3),
                         fill = 'tonexty',
                         fillcolor = '#8FB5FE',
                         showlegend=False
                        ))

# pronostico
fig.add_trace(go.Scatter(x=data_fig[data_fig.type=='pronostico'].index, 
                         y=data_fig[data_fig.type=='pronostico'].oni,
                         text=data_fig[data_fig.type=='pronostico'].oni.apply(lambda x: str(round(x,2)) ),
                         textposition="bottom right",
                         marker_symbol='star',
                         marker_line_width=3,
                         marker_size=3,
                         marker_line_color='#0057FF',
                         marker_color='#0057FF',
                         mode='lines+markers+text',
                         name='NIÑO3.4 NARX entrenamiento',
                         line=dict(color='#EF02F3', width=3),
                         showlegend=False
                        ))

fig.add_annotation(x=pd.Timestamp('2023-01-01'), y=0.5+0.2,text="débil",showarrow=False,yshift=10,font=dict(color='#FF6C6C') )
fig.add_hline(y=0.5, line_width=0.75, line_dash="dash", line_color="#FF6C6C")

fig.add_annotation(x=pd.Timestamp('2023-01-01'), y=1+0.2,text="moderado",showarrow=False,yshift=10,font=dict(color='#FF3F3F') )
fig.add_hline(y=1, line_width=1, line_dash="dash", line_color="#FF3F3F")

fig.add_annotation(x=pd.Timestamp('2023-01-01'), y=1.5+0.2,text="fuerte",showarrow=False,yshift=10,font=dict(color='#FF0000') )
fig.add_hline(y=1.5, line_width=1.25, line_dash="dash", line_color="#FF0000")

fig.add_annotation(x=pd.Timestamp('2023-01-01'), y=2+0.2,text="muy fuerte",showarrow=False,yshift=10,font=dict(color='#D70000') )
fig.add_hline(y=2, line_width=1.50, line_dash="dash", line_color="#D70000")


fig.add_hline(y=2.5, line_width=1.75, line_dash="dash", line_color="#AD0000")


fig.add_annotation(x=pd.Timestamp('2023-01-01'), y=-0.5-0.35,text="débil",showarrow=False,yshift=10,font=dict(color='#69A6FF') )
fig.add_hline(y=-0.5, line_width=0.75, line_dash="dash", line_color="#69A6FF")

fig.add_annotation(x=pd.Timestamp('2023-01-01'), y=-1-0.35,text="moderado",showarrow=False,yshift=10,font=dict(color='#6979FF') )
fig.add_hline(y=-1, line_width=1, line_dash="dash", line_color="#6979FF")

fig.add_annotation(x=pd.Timestamp('2023-01-01'), y=-1.5-0.35,text="fuerte",showarrow=False,yshift=10,font=dict(color='#3F53FF') )
fig.add_hline(y=-1.5, line_width=1.25, line_dash="dash", line_color="#3F53FF")

fig.add_annotation(x=pd.Timestamp('2023-01-01'), y=-2-0.35,text="muy fuerte",showarrow=False,yshift=10,font=dict(color='#001BFF') )
fig.add_hline(y=-2, line_width=1.5, line_dash="dash", line_color="#001BFF")

fig.add_hline(y=-2.5, line_width=1.75, line_dash="dash", line_color="#00059A")

# el nino y la nina
fig.add_annotation(x=pd.Timestamp('2019-01-01'), y=2+0.2,text="El Niño",showarrow=False,yshift=15,font=dict(color='#D70000') )
fig.add_annotation(x=pd.Timestamp('2019-01-01'), y=-2-0.35,text="La Niña",showarrow=False,yshift=15,font=dict(color='#001BFF') )

# linea de pronostico
fig.add_vline(x=data_fig[data_fig.type=='pronostico'].index.min(), line_width=3, line_dash="dash", line_color="#580606")

fig.update_xaxes(tickformat="%Y/%m",showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', gridcolor='#E4E4E4',mirror=True,
                 ticks="outside", tickwidth=2, tickcolor='#5C2B05', ticklen=10)

fig.update_traces(textfont_size=14)
fig.update_layout(title="""
                        Índice Niño Oceánico (ONI) pronóstico periodo {date_init} al {date_fin}
                        <br><sup>Promedio de 3-meses para las anomalías SST en la región Niño 3.4 (variación periodos base de 30-años)
                        </sup>
                        """.format(date_init=str(data_fig[data_fig.type=='pronostico'].index.min().strftime('%Y-%m-%d')),
                                   date_fin=str(data_fig[data_fig.type=='pronostico'].index.max().strftime('%Y-%m-%d')) ),
                  xaxis_title='Mes',
                  yaxis_title='Promedio 3-Meses anomalías SST (°C)',
                   uniformtext_minsize=8,
                   uniformtext_mode='hide',
                   height=800,
                   width=1500,
                   font = dict(size = 22),
                   xaxis_range=['2018-01-01','2023-06-01']
                 )

fig.show()